In [1]:
import torch
import lightning as pl
import torchmetrics.functional as metricfunctions
import torchvision.models as models

In [2]:
model = models.resnet50(models.ResNet50_Weights.IMAGENET1K_V2)

c:\Users\adity\.conda\envs\lightningML\lib\site-packages\torchvision\models\_utils.py:135: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.


In [3]:
x = torch.rand(7, 3, 224, 224)
y = model(x)
y.shape

torch.Size([7, 1000])

In [4]:
children_list = list(model.children())[:-1]

In [14]:
feature_ext = torch.nn.Sequential(*children_list)
feature_ext.eval()
feature_ext.requires_grad_(False)

y = feature_ext(x)
y.shape

torch.Size([7, 2048, 1, 1])

In [20]:
feature_ext

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [15]:
y = torch.flatten(y, 1)
y.shape

torch.Size([7, 2048])

In [17]:
a = torch.nn.Linear(2048, 10)(y)

a.shape

torch.Size([7, 10])

In [18]:
a

tensor([[-0.0530,  0.1222,  0.0752,  0.0423,  0.1954,  0.1212, -0.4607,  0.0977,
         -0.0191,  0.2776],
        [-0.0783,  0.1303,  0.0201,  0.0136,  0.1872,  0.0947, -0.4896,  0.0614,
         -0.0149,  0.2542],
        [-0.0585,  0.1048,  0.0442,  0.0411,  0.1976,  0.1062, -0.4876,  0.0674,
          0.0051,  0.2633],
        [-0.1028,  0.1168,  0.0222,  0.0119,  0.2059,  0.1052, -0.4531,  0.0645,
         -0.0067,  0.2621],
        [-0.0764,  0.0990,  0.0407,  0.0368,  0.1899,  0.1107, -0.4822,  0.0641,
         -0.0133,  0.2560],
        [-0.0516,  0.1027,  0.0254,  0.0449,  0.1913,  0.1286, -0.4861,  0.1037,
          0.0130,  0.2487],
        [-0.0678,  0.1202,  0.0237,  0.0323,  0.2086,  0.0855, -0.4472,  0.0967,
          0.0072,  0.2797]], grad_fn=<AddmmBackward0>)

In [19]:
a.argmax(dim=1)

tensor([9, 9, 9, 9, 9, 9, 9])

In [20]:
for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(model.fc.in_features, 10)
for i, param in enumerate(model.parameters()):
    print(f"param\t{i}\tshaped\t{param.shape}\t\t\trequires_grad = {param.requires_grad}")

param	0	shaped	torch.Size([64, 3, 7, 7])			requires_grad = False
param	1	shaped	torch.Size([64])			requires_grad = False
param	2	shaped	torch.Size([64])			requires_grad = False
param	3	shaped	torch.Size([64, 64, 1, 1])			requires_grad = False
param	4	shaped	torch.Size([64])			requires_grad = False
param	5	shaped	torch.Size([64])			requires_grad = False
param	6	shaped	torch.Size([64, 64, 3, 3])			requires_grad = False
param	7	shaped	torch.Size([64])			requires_grad = False
param	8	shaped	torch.Size([64])			requires_grad = False
param	9	shaped	torch.Size([256, 64, 1, 1])			requires_grad = False
param	10	shaped	torch.Size([256])			requires_grad = False
param	11	shaped	torch.Size([256])			requires_grad = False
param	12	shaped	torch.Size([256, 64, 1, 1])			requires_grad = False
param	13	shaped	torch.Size([256])			requires_grad = False
param	14	shaped	torch.Size([256])			requires_grad = False
param	15	shaped	torch.Size([64, 256, 1, 1])			requires_grad = False
param	16	shaped	torch.Size([64])

In [14]:
model.heads.head = torch.nn.Linear(
    in_features=model.heads.head.in_features,
    out_features=10,
    bias=True,
)

In [20]:
a = children_list[-2].children()